In [1]:
%reset
# del s, t, u, v, s_features, t_features, u_features, v_features

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
'''
    Usage: 
    1. Load the measurement ride which is a pickle
    2. Use defect library to process and get the featureset
    3. Save it perhaps
'''

import pandas as pd
import numpy as np
from src.utils.defect_utils import Featureset
import src.utils.import_pickle as ip
### 20190925 - calibrated - only defects
### 20190128 - insulation
"""
old_filenames=['gDFZ_2019-05-27T08_55_55_819Z_DFZ01', # s
               'gDFZ_2019-05-27T10_03_59_077Z_DFZ01', # t - worked with this
               'gDFZ_2019-05-27T13_05_53_330Z_DFZ01', # u
               'gDFZ_2019-05-27T14_10_51_425Z_DFZ01'] # v"""

new_filenames = ['__new_gDFZ_2019-05-27T08_55_55_819Z_DFZ01', # s - 819Z_DFZ01
                 '__new_gDFZ_2019-05-27T10_03_59_077Z_DFZ01', # t - 077Z_DFZ01
                 '__new_gDFZ_2019-05-27T13_05_53_330Z_DFZ01', # u - 330Z_DFZ01
                 '__new_gDFZ_2019-05-27T14_10_51_425Z_DFZ01'] # v - 425Z_DFZ01


s = ip.import_pickle(new_filenames[0]) # not that good
t = ip.import_pickle(new_filenames[1])
u = ip.import_pickle(new_filenames[2])
v = ip.import_pickle(new_filenames[3])

print("#################### S ####################")
s_features = Featureset(s)
print("#################### T ####################")
t_features = Featureset(t)
print("#################### U ####################")
u_features = Featureset(u)
print("#################### V ####################")
v_features = Featureset(v)

Importing Pickle: __new_gDFZ_2019-05-27T08_55_55_819Z_DFZ01.pickle
Warning, the name:  HOL 303.2 - 9005_D5.1  is tranformed to: 
HOL 303.2-9005D5.1
Warning, the name:  BN 42_K6.2 - 43.2  is tranformed to: 
BN 42K6.2-43.2
Warning, the name:  BN 42_K6.1-2  is tranformed to: 
BN 42K6.1-2
Warning, the name:  BN 38_K6.3 - 42.3  is tranformed to: 
BN 38K6.3-42.3
Warning, the name:  BN 38_K6.3-4  is tranformed to: 
BN 38K6.3-4
Warning, the name:  BN 41_K5.1 - 41.1  is tranformed to: 
BN 41K5.1-41.1
Warning, the name:  BN 41_K5.1-2  is tranformed to: 
BN 41K5.1-2
Warning, the name:  BN 15.3 - 41_K5.2  is tranformed to: 
BN 15.3-41K5.2
Importing Pickle: __new_gDFZ_2019-05-27T10_03_59_077Z_DFZ01.pickle
Importing Pickle: __new_gDFZ_2019-05-27T13_05_53_330Z_DFZ01.pickle
Importing Pickle: __new_gDFZ_2019-05-27T14_10_51_425Z_DFZ01.pickle


ZMON AXLE_11:   0%|          | 0/87 [00:00<?, ?it/s]

Warning, the name:  BN 2_K6.1-2  is tranformed to: 
BN 2K6.1-2
#################### S ####################


ZMON AXLE_11:   0%|          | 0/56 [00:00<?, ?it/s]

#################### T ####################


ZMON AXLE_11:   0%|          | 0/74 [00:00<?, ?it/s]

#################### U ####################


ZMON AXLE_11:   0%|          | 0/99 [00:00<?, ?it/s]

#################### V ####################


insulationjoint AXLE_42: 100%|██████████| 130/130 [00:19<00:00,  6.53it/s]


In [3]:
DEF = pd.concat([s_features.defects, t_features.defects, u_features.defects, v_features.defects ], axis=0)
SWI = pd.concat([s_features.switches, t_features.switches, u_features.switches, v_features.switches ], axis=0)
INS = pd.concat([s_features.ins_joints, t_features.ins_joints, u_features.ins_joints, v_features.ins_joints ], axis=0)

In [4]:
### Save
from src.utils.defect_utils import save_pickle
save_pickle([s_features, t_features, u_features, v_features], 'new2')

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pylab 

def plot_entity(df, threshold=None, entity= "", savepath='./AiyuDocs/new_signalplots/'):
    """
    Plots each entity and saves them to a figure
    params:
        df: dataframe for an entity
    """
    accs = np.asarray(np.vstack([val for val in df.accelerations]))
    ybot, ytop = np.amax(accs), np.amin(accs)

    feature_names  = df.columns
    display_params = feature_names[3:-1] # Skip acc, timestamp, window length
    for entityname, row in df.iterrows():
        fig = plt.figure(figsize=(20,5), constrained_layout=True)
        
        x = (row['timestamps'] - row['timestamps'][0])/10**9
        y = row['accelerations']
        

        ax1 = fig.add_axes((0.1, 0.2, 1, 1))
        ax1.plot(x, y, zorder = 1)
        if entity == 'defect':
            ax1.set_title("Defect: " + entityname)
        else:
            ax1.set_title(entityname)
            
        ax1.set_xlabel("Seconds")
        ax1.set_ylabel("Amplitude")
        ax1.set_ylim(ybot,ytop)
        ax1.margins(x=0, y=0.1)
        
        caption_list = []
        for feature in display_params:
            if feature == 'vehicle_speed(m/s)':
                caption_list.append('|%s|: %.1f'%(feature, df.loc[entityname, feature]))
            else:
                caption_list.append('|%s|: %s'%(feature, df.loc[entityname, feature]))
                
        if 'EKW' in entityname:
            entityname = 'EKW_DKW' + entityname[9:] 
            
        caption = ',    '.join(caption_list)
                                
        fig.text(0.1, .0005, caption, ha='left', fontsize=12)
        fig.set_size_inches(20, 5, forward=True)
        fig.savefig(savepath + entityname,  bbox_inches='tight')
        #plt.show()
        
    pylab.close('all')

In [7]:
plot_entity(INS)

/Users/Mac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: Calling figure.constrained_layout, but figure not setup to do constrained layout.  You either called GridSpec without the fig keyword, you are using plt.subplot, or you need to call figure or subplots with the constrained_layout=True kwarg.
/Users/Mac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [8]:
plot_entity(SWI)

/Users/Mac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: Calling figure.constrained_layout, but figure not setup to do constrained layout.  You either called GridSpec without the fig keyword, you are using plt.subplot, or you need to call figure or subplots with the constrained_layout=True kwarg.
/Users/Mac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [24]:
plot_entity(DEF, entity='defect')

/Users/Mac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:49: UserWarning: Calling figure.constrained_layout, but figure not setup to do constrained layout.  You either called GridSpec without the fig keyword, you are using plt.subplot, or you need to call figure or subplots with the constrained_layout=True kwarg.
/Users/Mac/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
